### First Part: O-RAN Data Loading and Pre-processing

The dataset is loaded from a pre-cleaned CSV file. Only two core system-level features are used as model inputs:
- `airtime`
- `selected_mcs`

The target variable is `pm_power`, representing power consumption.


### Environment Setup

This experiment is implemented using PyTorch. The following cell verifies the PyTorch version and whether GPU acceleration is available.


In [ ]:
import torch
print(torch.__version__)
print("CUDA available:", torch.cuda.is_available())


2.10.0+cpu
CUDA available: False


In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1) load data
df = pd.read_csv("clean_oran_stage1.csv")

feature_cols = ["airtime", "selected_mcs", "txgain"]

target_col = "pm_power"

### Handling Missing Values

Rows containing missing or non-numeric values in the selected features or target variable are removed to ensure data consistency.


In [ ]:
df = df.dropna(subset=feature_cols + [target_col]).copy()
for c in feature_cols + [target_col]:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=feature_cols + [target_col]).copy()

X = df[feature_cols].values
y = df[target_col].values

### Dataset Split

The dataset is split into training, validation, and test sets:
- 80% training + test split
- 10% of the training set is further used as a validation set

This ensures that model selection is performed using unseen validation data.


In [ ]:
# 2) split: train/test then train/val
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val,  y_train, y_val  = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### Feature Scaling

All input features are standardised using `StandardScaler`.  
The scaler is fitted only on the training data and then applied to validation and test sets.


In [ ]:
# 3) scale 
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s   = scaler.transform(X_val)
X_test_s  = scaler.transform(X_test)

print("Shapes:", X_train_s.shape, X_val_s.shape, X_test_s.shape)

Shapes: (12600, 3) (1400, 3) (3501, 3)


### PyTorch Dataset and DataLoader

A custom `Dataset` class is defined to convert the tabular data into PyTorch tensors.  
Mini-batches are generated using `DataLoader` with a batch size of 64.


In [ ]:

class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
    
    def __len__(self): 
        return len(self.X)
    
    def __getitem__(self, idx): 
        return self.X[idx], self.y[idx]

train_loader = DataLoader(TabularDataset(X_train_s, y_train), batch_size=64, shuffle=True)
val_loader   = DataLoader(TabularDataset(X_val_s, y_val), batch_size=64, shuffle=False)
test_loader  = DataLoader(TabularDataset(X_test_s, y_test), batch_size=64, shuffle=False)


### Baseline DNN Model

The baseline model is a fully-connected feed-forward neural network with the following architecture:

Input → 64 → 64 → 32 → Output

ReLU activations are applied after each hidden layer.


In [ ]:
class BaselineDNN(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 64), nn.ReLU(),
            nn.Linear(64, 64), nn.ReLU(),
            nn.Linear(64, 32), nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x): 
        return self.net(x)

### Training Setup

The model is trained using:
- Optimizer: Adam
- Learning rate: 0.001
- Loss function: Mean Squared Error (MSE)
- Number of epochs: 100


In [ ]:




def mean_relative_error(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return float(np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + eps)) * 100)


device = "cuda" if torch.cuda.is_available() else "cpu"
model = BaselineDNN(in_dim=len(feature_cols)).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

best_val = float("inf")
best_state = None



### Model Training and Validation

During training, both training loss and validation loss are monitored.  
The model state corresponding to the lowest validation MSE is saved.


In [ ]:

print("Oran traning:")

for epoch in range(1, 101):
    # train
    model.train()
    train_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        opt.step()
        train_loss += loss.item() * len(xb)
    train_loss /= len(train_loader.dataset)

    # val
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            val_loss += loss_fn(pred, yb).item() * len(xb)
    val_loss /= len(val_loader.dataset)

    if val_loss < best_val:
        best_val = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d} | train MSE {train_loss:.6f} | val MSE {val_loss:.6f}")

# load best

model.load_state_dict(best_state)
print("Best val MSE:", best_val)


Oran traning:
Epoch 001 | train MSE 49.971478 | val MSE 3.363712
Epoch 010 | train MSE 0.109962 | val MSE 0.114097
Epoch 020 | train MSE 0.109729 | val MSE 0.104140
Epoch 030 | train MSE 0.107123 | val MSE 0.109296
Epoch 040 | train MSE 0.110020 | val MSE 0.102672
Epoch 050 | train MSE 0.110029 | val MSE 0.106829
Epoch 060 | train MSE 0.107830 | val MSE 0.105004
Epoch 070 | train MSE 0.107610 | val MSE 0.106051
Epoch 080 | train MSE 0.104870 | val MSE 0.104004
Epoch 090 | train MSE 0.104970 | val MSE 0.103784
Epoch 100 | train MSE 0.105604 | val MSE 0.098335
Best val MSE: 0.09833530570779528


### Evaluation Metrics

The trained model is evaluated on the test set using the following metrics:
- Mean Squared Error (MSE)
- Root Mean Squared Error (RMSE)
- Mean Absolute Error (MAE)
- Mean Relative Error (MRE)


In [ ]:
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        pred = model(xb).cpu().numpy().reshape(-1)
        y_pred.append(pred)
        y_true.append(yb.numpy().reshape(-1))

y_true = np.concatenate(y_true)
y_pred = np.concatenate(y_pred)

mse  = mean_squared_error(y_true, y_pred)
rmse = float(np.sqrt(mse))
mae  = mean_absolute_error(y_true, y_pred)
mre  = mean_relative_error(y_true, y_pred)

print("=== O-RAN Baseline DNN (A→A) ===")
print("X:", feature_cols, " y:", target_col)
print(f"MSE  : {mse:.6f}")
print(f"RMSE : {rmse:.6f}")
print(f"MAE  : {mae:.6f}")
print(f"MRE% : {mre:.4f}")


=== O-RAN Baseline DNN (A→A) ===
X: ['airtime', 'selected_mcs', 'txgain']  y: pm_power
MSE  : 0.098505
RMSE : 0.313855
MAE  : 0.243016
MRE% : 1.8181
